# Strands Agents with Amazon Bedrock AgentCore Runtime


## Overview

In this tutorial we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. 
For demonstration purposes, we will  use a Strands Agent using Amazon Bedrock models

In our example we will use a very simple agent with tools like: `file_read`, `file_write`, `speak`

### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Amazon Bedrock models
* Using Strands Agents

## Prerequisites

To execute this tutorial you will need:
 * Python 3.10+
 * AWS credentials
 * Amazon Bedrock AgentCore SDK
 * Strands Agents
 * Docker running
 
First, install the required dependencies by running this command in your terminal:
 
```bash
$ uv add -r requirements.txt
```
 
This will install all the necessary packages listed in the requirements.txt file using the uv package manager.

## Creating your agents and experimenting locally


Before we deploy our agents to AgentCore Runtime, we need to modify our existing application. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. 

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

```python
from bedrock_agentcore import BedrockAgentCoreApp
from strands import Agent
from strands_tools import file_read, file_write, speak

# Step 1: Define the app 
app = BedrockAgentCoreApp()

# Step 2: Define the system prompt
system_prompt = """YOUR_SYSTEM_PROMPT"""

# Step 3: Define the agent
agent = Agent(
    system_prompt=system_prompt,
    tools=[
        file_read,
        file_write,
        speak,
    ],
)

@app.entrypoint
def invoke(payload):
    """Process user input and return a response"""
    user_message = payload.get("prompt", "Hello")
    result = agent(user_message)
    return {"result": result.message}

if __name__ == "__main__":
    app.run()


## Under the hood

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime (using `agentcore cli`)

### **Step 1:** Configure the agent using `agentcore configure`

```bash
$ agentcore configure --entrypoint app.py \
                    --execution-role arn:aws:iam::507922848584:role/genesis-pa-runtime-01-role \
                    --requirements-file requirements.txt \
                    --ecr auto \
                    --disable-otel \
                    --name my_demo_agent_01
```

# <div align="center"><img src="../imgs/agentcore_configure.png" width="1000"/></div>

### **Step 2:** Launch the agent using `agentcore launch`

```bash
$ agentcore launch
```
# <div align="center"><img src="../imgs/agent_launch_2.png" width="1200"/></div>

At this point in time, your agentic application is pushed to ECR 

# <div align="center"><img src="../imgs/agent_launch_0.png" width="800"/></div>


### **Step 3:** Launch the agent using `agentcore invoke`


```bash
$ agentcore invoke '{"prompt": "list all files in the current directory"}'
```

# <div align="center"><img src="../imgs/agent_invoke.png" width="1200"/></div>


## Invoke the agent (using `boto3`)

### **Step 0:** Create Bedorck AgentCore Client

In [1]:
import boto3
import json
import uuid

region = 'us-east-1'

# Create a client for the control plane
bedrock_agent_core = boto3.client('bedrock-agentcore-control', region_name=region)

# Create a client for the runtime
bedrock_agent_core_runtime = boto3.client('bedrock-agentcore', region_name=region)

# <div align="center"><img src="../imgs/agent_invoke_boto3.png" width="1200"/></div>


### **Step 1:** List the agents

In [2]:
bedrock_agent_core.list_agent_runtimes()['agentRuntimes']

[{'agentRuntimeArn': 'arn:aws:bedrock-agentcore:us-east-1:507922848584:runtime/my_demo_agent_01-R459msC7UO',
  'agentRuntimeId': 'my_demo_agent_01-R459msC7UO',
  'agentRuntimeVersion': '1',
  'agentRuntimeName': 'my_demo_agent_01',
  'lastUpdatedAt': datetime.datetime(2025, 7, 17, 17, 18, 31, 171955, tzinfo=tzutc()),
  'status': 'READY'},
 {'agentRuntimeArn': 'arn:aws:bedrock-agentcore:us-east-1:507922848584:runtime/my_demo_agent_02-EBURnU4V7T',
  'agentRuntimeId': 'my_demo_agent_02-EBURnU4V7T',
  'agentRuntimeVersion': '1',
  'agentRuntimeName': 'my_demo_agent_02',
  'lastUpdatedAt': datetime.datetime(2025, 7, 17, 17, 47, 59, 734599, tzinfo=tzutc()),
  'status': 'READY'},
 {'agentRuntimeArn': 'arn:aws:bedrock-agentcore:us-east-1:507922848584:runtime/my_demo_agent_03-AXarlFDh6I',
  'agentRuntimeId': 'my_demo_agent_03-AXarlFDh6I',
  'agentRuntimeVersion': '1',
  'agentRuntimeName': 'my_demo_agent_03',
  'lastUpdatedAt': datetime.datetime(2025, 7, 17, 17, 46, 59, 673065, tzinfo=tzutc()),

### **Step 2:** Select an Agent of your choice 

In [3]:
from utills import select_agent_runtime

# Step 1: Get the list of agent runtimes
agent_runtimes = bedrock_agent_core.list_agent_runtimes()['agentRuntimes']
arn_widget = select_agent_runtime(agent_runtimes)

Output()

In [4]:
myagentRuntimeArn = arn_widget.value
print(f"Your selected agent runtime ARN is: {myagentRuntimeArn}")

Your selected agent runtime ARN is: arn:aws:bedrock-agentcore:us-east-1:507922848584:runtime/my_demo_agent_01-R459msC7UO


### **Step 3:** Invoke the agent

In [5]:
runtimeSessionId=str(uuid.uuid4())

response = bedrock_agent_core_runtime.invoke_agent_runtime(
    agentRuntimeArn=myagentRuntimeArn,
    qualifier="DEFAULT",
    runtimeSessionId=runtimeSessionId,
    payload = json.dumps({"prompt": "Hi, how are you ?"}).encode()
)

In [6]:
from utills import process_output
process_output(response)

### 🤖 Bedrock AgentCore Response
Hello! I'm doing well, thank you for asking. I'm your helpful assistant ready to help you with various tasks like reading and writing files, summarizing content, or even converting text to speech. 

Is there something specific I can help you with today? Perhaps you'd like me to:
- Read or summarize a document for you
- Create a new file with some content
- Check what files are in a directory
- Or assist with something else?

Just let me know what you need!

### **Step 4:** Chat based invocation

In [7]:
while True:
    user_input = input("\n🧑 You: ").strip()
    if user_input.lower() in {"exit", "quit"}:
        print("Goodbye!")
        break
    if not user_input:
        continue
    
    print("🤖 Bedrock AgentCore: ")
    # Invoking the agent
    response = bedrock_agent_core_runtime.invoke_agent_runtime(
        agentRuntimeArn=myagentRuntimeArn,
        qualifier="DEFAULT",
        runtimeSessionId=runtimeSessionId,
        payload = json.dumps({"prompt": user_input}).encode()
    )
    process_output(response, chat_mode=True)

🤖 Bedrock AgentCore: 


<div style="background-color:#f0f8ff; padding:10px; border-radius:8px; border-left:4px solid #4682B4;">
🤖 <strong>Bedrock AgentCore</strong>:<br><br>Hello again! I'm here and ready to help you with any tasks you might have. If you have any files you'd like me to read, content you'd like me to create, or any other assistance you need, please let me know. I'm ready whenever you are.
</div>

🤖 Bedrock AgentCore: 


<div style="background-color:#f0f8ff; padding:10px; border-radius:8px; border-left:4px solid #4682B4;">
🤖 <strong>Bedrock AgentCore</strong>:<br><br>I have access to the following tools to help you:<br><br>1. **file_read**: This tool allows me to read and interact with files in various ways:<br>   - View file contents<br>   - Search for specific text in files<br>   - List files matching specific patterns<br>   - Read specific line ranges<br>   - Show file statistics<br>   - Preview file contents<br>   - Compare files (diff)<br>   - View version history<br>   - Generate document blocks for various file formats (PDF, CSV, DOC, etc.)<br><br>2. **file_write**: I can create new files or write content to existing files.<br><br>3. **speak**: I can convert text to speech, either using:<br>   - The fast "say" command on macOS<br>   - Amazon Polly for higher quality speech on other operating systems<br><br>Would you like me to demonstrate any of these tools or help you with a specific task?
</div>

🤖 Bedrock AgentCore: 


<div style="background-color:#f0f8ff; padding:10px; border-radius:8px; border-left:4px solid #4682B4;">
🤖 <strong>Bedrock AgentCore</strong>:<br><br>Here are the files in your present directory:<br>1. `__pycache__/app.cpython-313.pyc` - A Python compiled bytecode file<br>2. `app.py` - A Python script file<br>3. `requirements.txt` - Typically contains Python package dependencies<br>4. `runtime_with_strands_and_bedrock_models.ipynb` - A Jupyter notebook file<br><br>Would you like me to examine any of these files in more detail?
</div>

🤖 Bedrock AgentCore: 


<div style="background-color:#f0f8ff; padding:10px; border-radius:8px; border-left:4px solid #4682B4;">
🤖 <strong>Bedrock AgentCore</strong>:<br><br>Goodbye! It was nice chatting with you. If you need any assistance in the future with reading files, writing content, or any other tasks, feel free to reach out. Have a great day!
</div>

Goodbye!
